# 1. Thread là gì 

# 1.1 Cấu trúc của một thread

Thead là đơn vị nhỏ nhất của process có thế thực thi. Nếu như 1 process chỉ có 1 thread (**main thread**) thì nó được coi như là **single-threaded process**.

![image.png](https://kuleuven-diepenbeek.github.io/osc-course/img/os/db_multithreaded_v2.png)

🟦 Vùng dùng chung của tiến trình (Process-wide, được chia sẻ bởi tất cả các thread):
1. Code (mã lệnh)
- Chứa mã lệnh của chương trình (chẳng hạn như các hàm, vòng lặp, biểu thức).
- Đây là phần không thay đổi (read-only), để đảm bảo an toàn khi nhiều thread cùng chạy.
2. Heap (bộ nhớ cấp phát động)
- Dùng để cấp phát bộ nhớ động trong runtime (qua new, malloc, v.v.).
- Tất cả các thread trong cùng một tiến trình đều có thể truy cập vùng heap.
Ví dụ: bạn tạo một danh sách liên kết tại runtime thì danh sách này nằm ở heap.
3. Files (tập tin)
- Đại diện cho các tài nguyên hệ thống mà tiến trình đang mở, ví dụ như file, socket, thiết bị I/O.
- Cũng được chia sẻ bởi các thread trong tiến trình.

🟥 Vùng riêng của thread (Thread-specific):

4. Registers (thanh ghi)
- Là các thanh ghi CPU lưu trữ dữ liệu tạm thời cho thread, ví dụ: kết quả trung gian của phép tính.
- Mỗi thread có tập thanh ghi riêng, không chia sẻ.

5. Program Counter (bộ đếm chương trình)
- Ghi lại địa chỉ lệnh tiếp theo mà thread sẽ thực hiện.
- Mỗi thread có bộ đếm riêng, giúp thread biết mình đang thực thi đến đâu trong chương trình.
6. Stack (ngăn xếp)
- Dùng để lưu các lời gọi hàm (call stack), biến cục bộ, và thông tin trả về.
- Mỗi thread có một stack riêng. Khi gọi hàm đệ quy, hay tạo biến nội bộ trong hàm, chúng sẽ nằm ở đây.
7. Thread (luồng)
- Đại diện cho một luồng điều khiển (execution flow) trong tiến trình,
- Trong tiến trình đơn luồng, chỉ có một thread thực thi mọi thứ.

# 1.2 Tương tác giữa các thread

Khải niệm **Concurrency** (hay tiếng Việt là tính đồng thời) là một khái niệm trong khoa học máy tính chỉ việc nhiều tác vụ có thể tiến hành trong cùng một khoảng thời gian. Tuy nhiên, không nhất thiết tất cả các tác vụ đó phải thực sự chạy cùng lúc (**song song**), mà chỉ cần chúng có thể tiến hành **xen kẽ** nhau.

![Program Diagram](https://gitlab.com/ltpho448/learnCPPwithSharko/-/raw/main/Assets/thread3.png)

Điều kiện trên là máy tính chính ta chỉ có 1 core, core có 2 thread. Như hình minh họa thì **thread1** chạy những khoảng thời gian t1,t3 trong khi đó **thread2** sẽ chạy những khoảng thời gian t2,t4  .Vậy là 2 thread này sẽ chạy những khoảng xen kẽ nhau chứ bản chất chúng không thực sự song song. Thread chỉ song song thực sự khi 2 thread đó chạy trên 2 core.

![Program Diagram](https://gitlab.com/ltpho448/learnCPPwithSharko/-/raw/main/Assets/multithread1.png)
>Các thread sẽ chia sẻ tất cả dữ liệu với nhau. Mỗi một thread đều có một stack riêng của chúng. Và chúng đề có stack counter lưu lại program register câu lệnh tiếp theo thread đó phải thực hiện.

![Program Diagram](https://gitlab.com/ltpho448/learnCPPwithSharko/-/raw/main/Assets/multithread2.png)
> Mỗi thread đều có trỏ đến vùng text code để thực thi chương trình

# 2. Tại sao chúng ta lại cần multithread

- Tính đáp ứng(**Responsiveness**): Khi một chương trình được chia thành nhiều luồng (threads), trải nghiệm người dùng sẽ cảm thấy mượt mà hơn. Các luồng chuyên biệt có thể được tạo để xử lý yêu cầu của người dùng và cung cấp phản hồi (hình ảnh), trong khi các luồng khác có thể, ví dụ, xử lý dữ liệu ở chế độ nền.

- Chia sẻ tài nguyên(**Resource sharing**): Các chương trình có nhiều luồng thường cần một số hình thức giao tiếp giữa các luồng này. Việc này được thực hiện dễ dàng hơn giữa các luồng so với giữa các tiến trình (processes), vì các luồng mặc nhiên chia sẻ bộ nhớ qua heap của chúng. Điều này sẽ được thảo luận chi tiết trong phần tiếp theo.

- Tính chi phí(**Economy**): Việc chuyển đổi ngữ cảnh (context switching) trở nên ít tốn kém hơn khi chuyển đổi giữa các luồng so với giữa các tiến trình. Điều này là do trạng thái của mỗi luồng cần theo thdõi ít hơn so với trạng thái của mỗi tiến trình.

- Khả năng mở rộng(**Scalability**): Nhiều luồng có thể chạy song song trên các hệ thống đa lõi, trái ngược với một tiến trình đơn luồng.

# 2.1 Amdahl’s law

![image.png](https://kuleuven-diepenbeek.github.io/osc-course/img/os/serial_vs_parallel.png)

Như vậy chúng ta thấy nếu ứng chương trình ta sử dụng càng nhiều multithread thì chương trình chúng ta càng nhanh. Hay chúng ta mong đợi rằng program chúng ta chạy trên N core sẽ nhanh hơn gấp N lần so với program của chúng ta khi chạy trên 1 core. Điều này thực sự không đúng. Program của chúng ta chỉ thực sự chạy nhanh hơn N lần so với 1 core những tác vụ có thể song song được. 

Công thức tính cho tỉ lệ tăng tốc được: Speedup < 1 / (S + (1-S)/N). Trong đó:
- **S** là khoảng tỉ lệ thời gian program chạy tuần tự
- **N** là khoảng tị lệ thời gian program chạy song song

# 2.2 User threads vs System threads

![image.png](https://kuleuven-diepenbeek.github.io/osc-course/img/os/db_threads.png)

Hệ điều hành(**Operator system**) của chúng ta chạy rất nhiều chương trình(**program**). Mỗi chương trình có **ít nhất 1 process** hoặc nhiều hơn. Mỗi một process lại cũng có **ít nhất một Main Thread**. Trong một khoảng thời gian thì **OS** của chúng ta chạy khoảng 10-50 process, **10000** thread, trong đó thì máy tính chúng ta thường chỉ có khoảng 4, 6, 8 core(8, 12, 16 kernel thread). Có cách nào để máy  tính chúng ta ta có thể chạy được số lượng lớn process, thread đến vậy. Sử dụng cơ chế map giữa **user thread** và **kernel thread** nhiều cách (1 vs 1, 1 vs many, many vs many). Việc map này cũng có thể thay chuyển đổi giữa các CPU khác nhau việc này **OS** kiểm soát. Như vậy là 1 program của chúng ta gồm nhiều user thread tuy nhiên có thể chỉ được map với cùng 1 kernel thread thì program của chúng ta các thread chỉ coi như chạy tuần tự. 

# 3. Thực hành tạo thread

```c

#include <stdio.h>
#include <string.h>
#include <pthread.h>
#include <stdlib.h>
#include <unistd.h>

int counter;

void* startThreadJob() {
    unsigned long i = 0;
    counter += 1;

    printf("  Job %d started\n", counter);
    for(i=0; i<(0xFFFFFFFF); i++) {
        // do nothing
        // this is just here to keep the thread running for a while!
    }
    printf("  Job %d finished\n", counter);

    return NULL;
};

int main(void) {
  int i = 0, err;
  pthread_t tid[2];
  counter = 0;

  while(i < 2) {

    // pthread_create has 4 parameters:
    // 1. pointer to pthread_t, needed to keep thread state
    // 2. configuration arguments for the thread (passing NULL means we use the defaults here)
    // 3. pointer to the function that will run in a separate thread
    // 4. parameters to pass to the thread (no arguments for startThreadJob, so we pass NULL)

    err = pthread_create( &(tid[i]), NULL, &startThreadJob, NULL );

    if (err != 0) {
      printf("\ncan't create thread :[%s]", strerror(err));
    }

    i++;
  }

  // pthread_join pauses the current thread until the thread in the first argument is terminated
  // Note that the "main" function automatically also executes in a thread, which we often call the "main thread".
  // If the joined thread was already terminated, pthread_join returns immediately
  // The second argument can be used to store the return value of the thread
  // Our thread currently doesn't return anything, so we pass NULL

  pthread_join(tid[0], NULL);
  pthread_join(tid[1], NULL);

  return 0;
}
```

Kết quả chạy như sau 
```bash
dawn@dawn-Precision:~/Documents/C++$ cd "/home/dawn/Documents/C++/" && gcc tes.c -o tes && "/home/dawn/Documents/C++/"tes
  Job 1 started
  Job 2 started
  Job 2 finished
  Job 2 finished

```

# 4. Cách thread giao tiếp với nhau

## 4.1 Race condition

Như kết quả chúng ta thấy ở trên thì khi job 1 finish thì counter lại in ra thành 2. Việc này do hiện tượng **race conditon**.

```c
// Code that both threads will execute:
counter += 1;
local_id = counter;
// -----------------------------------

// Thread 1
counter += 1; // after this, counter == 1
    // Before Thread 1 can execute local_id = counter, it is paused by the CPU
    // Thread 2
    counter += 1; // after this, counter == 2
    local_id = counter; // thus, in Thread 2, local_id is correctly 2
// Here, the CPU switches back to Thread 1
// Thread 1
local_id = counter; // now, local_id is still 2, instead of 1... mission failed

```

> Bên cạnh đó còn có khái niệm quan trọng là **critical section** đây là vùng dữ liệu quan trọng mà multithread truy cập vào 

Trước khi đi vào các cách mà thread giao tiếp với nhau ta tìm hiểu nguyên nhân đăng sau hiện tượng **race condition**. Và để thực hiện một phép toán counter =  counter + 1; thì CPU phải thực hiện một chuỗi các instrions **non atocmic**:
1. Fetch value of counter from RAM and store it in the CPU cache memory
2. Fetch value from CPU cache memory and store it in a CPU register for the calculation
3. Add 1 to the register value
4. Write the register value back to the CPU cache
5. Write the cached value back to the RAM

Hoăc viết đơn giản lại gộp các instruction của CPU lại được:

```c
// In simple pseudocode, counter += 1 might look like this:
int temp = counter + 1; // temp is for example the CPU register here, and "counter" is the value in memory
counter = temp; // the register value is written back out to the memory
```

Dưới đây là một minh họa cho nguyên nhân hiện tượng **race condition** xảy ra chuỗi sự kiện:

```c
// Code that both threads will execute:
int temp = counter + 1;
counter = temp;
// -----------------------------------

// Thread 1
int temp = counter + 1; // after this, Thread 1's temp register contains the value 1. The "counter" value in RAM is still 0.
    // Before Thread 1 can actually store this temporary register result in memory, the CPU gives control to Thread 2
    // Thread 2
    int temp = counter + 1; // "counter" was still 0 in RAM, so Thread 2's temp register now also contains the value 1
    counter = temp; // The "counter" value in RAM is now 1
// Here, the CPU switches back to Thread 1
// Thread 1
counter = temp; // The "counter" value in RAM is still 1

```


Để giải quyết vấn đề này **race condition** có các phương pháp sau: 
- **Atomic operator**
- **Mutex/Lock**
- **Semaphore/Signal**

## 4.2 Atomic operator

Atomic operator được support bởi câu lệnh của CPU thay thì thực hiện một loạt chuỗi hành động như load data từ ram vào cache cpu, cpu- >register ... bla bla thì nó phép toán dùng **các instruction không thể bị cắt**. Khi một atomic operation được thực hiện, nó đảm bảo rằng:
- Toàn bộ thao tác hoàn thành hoặc không thực hiện gì (không có trạng thái nửa vời).
- Không có thread/process nào khác có thể can thiệp vào quá trình này (tránh race condition).

### 4.2.1 Atomic Operations được CPU hỗ trợ như thế nào?

**a. Single Instruction Atomicity**

Một số atomic operations được thực hiện bằng một lệnh CPU duy nhất, ví dụ:

x86:

LOCK XADD (atomic add và trả về giá trị cũ).

LOCK CMPXCHG (atomic compare-and-swap).

ARM:

LDREX/STREX (load/store với cơ chế exclusive access).

RISC-V:

AMOADD (atomic memory add).

Ví dụ:
Khi bạn gọi std::atomic<int>::fetch_add(1) trong C++, trình biên dịch sẽ sinh mã assembly tương ứng với LOCK XADD (trên x86), thực hiện cộng nguyên tử giá trị vào bộ nhớ mà không cần qua các bước trung gian như load vào register.

**b. Multi-Instruction Atomicity**

Một số atomic operations phức tạp hơn (ví dụ: read-modify-write) có thể yêu cầu nhiều lệnh CPU, nhưng CPU vẫn đảm bảo tính atomic thông qua:

Bus Locking: Khóa bus bộ nhớ để ngăn các core/thread khác truy cập.

Cache Coherency Protocol: Đảm bảo các core chỉ thao tác trên bản sao độc quyền của dữ liệu trong cache (ví dụ: MESI protocol).

# 4.2.2 Tại sao không cần load data từ RAM vào register?

**Cache là trung gian**: Dữ liệu thường nằm trong cache (L1/L2/L3) của CPU, không phải RAM. Khi thực hiện atomic operation, CPU thao tác trực tiếp trên cache với cơ chế **exclusive access** (đảm bảo không có core nào khác cùng truy cập).

**Hardware Guarantee**: CPU đảm bảo atomic operation được thực hiện trực tiếp trên địa chỉ bộ nhớ (không qua register) nếu dùng lệnh hỗ trợ sẵn.


# 4.2.3 Khi nào atomic operations KHÔNG dùng single instruction?
**Kiến trúc CPU cũ**: Trên các CPU không hỗ trợ lệnh atomic cho thao tác phức tạp, trình biên dịch phải dùng software-level locking (mutex) để mô phỏng atomicity.

**Custom Data Structures**: Khi thao tác trên struct phức tạp (không phải Trivially Copyable), bạn phải tự đồng bộ bằng mutex.



# 4.2.4 Ưu điểm và hạn chế của Atomic Operations

Ưu điểm
- Hiệu suất cao: Tránh được chi phí lock/unlock của mutex.
- Lock-Free Programming: Cho phép xây dựng cấu trúc dữ liệu không cần khóa (ví dụ: lock-free queue).

Hạn chế
- Giới hạn kiểu dữ liệu: Chỉ áp dụng được cho các kiểu dữ liệu đơn giản (int, pointer, struct Trivially Copyable).
- Platform Dependency: Các lệnh atomic phụ thuộc vào kiến trúc CPU.


# 4.3 Mutex và Locking

Để giải quyết bài toán của **operator atomic** đối với **critical sections** chỉ áp dụng với các dữ liệu đơn giản thì người ta sử dụng 1 cách khác cũng vô cùng hiệu quả là mutex/locking.

Ý tưởng thực hiện là với thread đang chuẩn bị truy cập vào vùng **critical section** thì ta sẽ thực hiện khóa dữ liệu đó và chỉ cho thread đó thao tác trên vùng **critical section**. Sau khi thực hiện xong thì nhả khóa cho phép các thread khác có thể truy cập vào vùng dữ liệu **critical section**. 

```c
pthread_mutex_t lock; // Khai báo mutex lock

void* thread_func(void* arg) {
    pthread_mutex_lock(&lock);   // Khóa critical section
    // ... Thao tác trên dữ liệu shared ...
    pthread_mutex_unlock(&lock); // Nhả khóa
    return NULL;
}
```

Tuy nhiên mutex/locking gặp phải một vấn đề đó là **DeadLock**. Giả sử có 2 thread trong đó, thread1 thực hiện kế nối với mạng -> đọc dữ liệu vào ổ cứng, thread2 đọc dữ liệu từ ổ cứng để gửi vào mạng. Hai thread nàỷ lần lượt sử dụng những liệu đối ngược nhau và để tránh **Race condition**, 2 thread này đều lần lượt thực hiện khóa trên các tài nguyên mà chúng mong muốn.
``` c
// Thread 1 wants to first read from the network, then write result to the hard disk
// Thread 2 wants to first read the contents of a file, then send it on the network

// Thread 1:
network.lock()
    // CPU pauses Thread 1 and activates Thread 2
    harddisk.lock()
// CPU pauses Thread 2 and activates Thread 1 again
harddisk.lock() // this will not complete, because Thread 2 is holding this lock already. Thread 1 has to wait
    // CPU pauses Thread 1 because it has to wait and activates Thread 2 again
    network.lock() // this will not complete, because Thread 1 is holding this lock already. Thread 2 has to wait

// Both threads are waiting endlessly AND also locking the network and the harddisk for any other threads that might arrive
```
> Cả 2 thread này sẽ đợi lẫn nhau, thead1 đợi harddick, thread2 đợi network ---> **Dead Lock**
>
> Để giải quyết đương đầu với bài toán **Dead Lock** người ta sử dụng một techique khác đó là **semaphores


# 4.4 Semaphores

# 4.4.1 Producer-Consumer Problems

Kỹ thuật **semaphores(signal)** này được áp dụng cho vấn đề **Producer-Consumer**. Trong đó một hoặc nhiều **thread producer** sẽ liên tục đưa dữ liệu vào trong **pipe(queue)**, trong khi tương tự có một hoặc nhiều **thread consumer** sẽ có nhiệm vụ nhận là xử lý dữ liệu. Đây là cũng một cách xử lý với trường hợp chúng ta phải cấp phát một vùng lớn bộ nhớ để các thread có thể *giao tiếp* được với nhau.

![image.png](https://kuleuven-diepenbeek.github.io/osc-course/img/Multi-Producers-Multi-Consumers.jpg)

```c
// PSEUDOCODE!
mutex myLock;
int items_in_array = 0; 
int* array = malloc(sizeof(int) * 10);

void* producer() 
{
    while (true) 
    {
        bool itemStored = false;
        Item item = produceItem();

        while( !itemStored ) {

            lock( myLock );
            
            if( items_in_array < 10 ) {
                array[ items_in_array ] = item;
                items_in_array += 1;

                itemStored = true;
            }

            unlock( myLock );
        }
    }
}
```


```c
// PSEUDOCODE!




void* consumer() 
{
    while (true) 
    {
        Item item = NULL;

        lock( myLock );
        
        if( items_in_array > 0 ) {
            item = array[ items_in_array - 1 ];
            items_in_array -= 1;
        }

        unlock( myLock );

        if( item != NULL ) {
            consumeItem(item);
        }
    }
}


```

Trên là ví dụ minh họa cho thiết lập mô hình **producer-consumer**. Một bên liên tục tạo ra dữ liệu, một bên liên tục tiêu thụ dữ liệu. Với các triển khai này, các thread liên tục kiểm tra xem khóa có đang free không để có thể lock. Tuy nhiên nếu một trong hai giai đoạn này kéo lại lâu hơn dự kiến. Ví dụ **producer** để tạo ra 1 Item item = produceItem(); tốn mất 20s trong khi đó **consumer** consumeItem(item);chỉ tốn mất 0.1s, và nếu thread1 đã tạo ta 1 Item rồi lại tiếp tục thực hiện lock để tạo ra 1 item tiếp theo. Có nghĩa thread consumer phải đợi tiếp ~20s chờ đợi để có cơ hội tiêu thụ 1 Item. Lúc đó **thread consumer** sẽ ở trạng thái **busy waiting** hoặc **polling**.

>Để giải quyết bài toán **thread** liên tục kiểm tra thì ta có kỹ thuật cho phép **thread** chờ sẽ theo dõi, nhận tín hiệụ từ các **thread** khác khi hoàn thành xong công việc

# 4.4.2 Semaphore concepts

Semaphrore bản chất là hoạt động theo một số đếm **counter**, nhưng có thể **automatically pause/resume threads that use** nếu khi **counter** này = 0 hoặc > 0.
This is typically done using three conceptual functions on the semaphore:
- init( mySemaphore, count ) : Start the semaphore value at count (can be either 0 or a positive number).
- wait( mySemaphore ) : Continues if mySemaphore is > 0 and decrements it by 1. If not, it pauses the current thread until mySemaphore becomes positive.
- post( mySemaphore ) : Increments mySemaphore by 1. This then automatically causes 1 of the waiting threads (if any) to be unpaused.

``` c
// PSEUDOCODE!
mutex myLock;

semaphore fill_count;  
init(fill_count,  0);

semaphore empty_count; 
init(empty_count, 10);

int items_in_array = 0; 
int* array = malloc(sizeof(int) * 10);

void* producer() 
{
    while (true) 
    {
        Item item = produceItem();

        wait( empty_count ); // THIS IS NEW!

        lock( myLock );
        
        array[ items_in_array ] = item;
        items_in_array += 1;

        unlock( myLock );
        
        post( fill_count ); // THIS IS NEW!
    }
}

```

```c
void* consumer() 
{
    while (true) 
    {
        Item item = NULL;

        wait( fill_count ); // THIS IS NEW! 

        lock( myLock );
        
        item = array[ items_in_array - 1 ];
        items_in_array -= 1;

        unlock( myLock );

        post( empty_count ); // THIS IS NEW! 

        consumeItem(item);
    }
}
```



Đọc xong đoạn mã sử dụng semaphores, câu hỏi đặt ra nếu đã dùng semaphores rồi thì tại sao vẫn cần sử dụng mutex bên trong ?

Câu trả lời là việc sử dụng semaphore là vẫn để cho một số lương các thread có thể đồng thời truy cập vào vùng **critical section** và chúng thực hiện khóa là để tránh các thread này gây ra hiện tượng **race condition** như đã trình bày ở trên. Ý nghĩa của việc sinh ra **semaphore** không phải để bỏ việc sử dụng **mutex** mà thực ra là ta muốn bỏ đi tình trạng **busy wating, pooling** do lòng lặp **while()** gây ra.

 Tóm tắt bản chất

| **Cơ chế**       | **Mục đích**                                                                 | **Ưu điểm**                                  |
|------------------|-----------------------------------------------------------------------------|---------------------------------------------|
| **Semaphore**    | Kiểm soát số lượng thread truy cập tài nguyên, loại bỏ busy-waiting.         | Tiết kiệm CPU, hỗ trợ đa luồng đồng thời.   |
| **Mutex**        | Đảm bảo exclusive access khi thao tác trên shared memory.                    | Ngăn race condition, đơn giản.              |
| **Kết hợp**      | Semaphore quản lý số lượng thread + Mutex bảo vệ dữ liệu.                    | Tối ưu hiệu suất và an toàn dữ liệu.        |

→ **Semaphore sinh ra để giải quyết busy-waiting, không phải để thay thế Mutex!**